In [1]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
#import Excel
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime

In [5]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
wb = xw.Book('Salt.xlsm')
setup = wb.sheets('Setup')
output = wb.sheets('Output')
strats = wb.sheets('Strat Tables')
performance = wb.sheets('Performance')

# Path For the Current deal that is selected based on Setup
directory = setup['Setup.Directory'].value
shelf = setup['Setup.Shelf'].value
deal = setup['Setup.Deal'].value
report = setup['Setup.Report'].value
num_Deals = setup['Setup.Num_Deals'].value
max_Report = setup['Setup.Max_Report_Num'].value

In [3]:
tapes = np.array(setup['Tapes'].value)
tapes = tapes[0:int(max_Report),0:int(num_Deals)]
dfTapes = pd.DataFrame(data=tapes[1:,1:], columns=tapes[0,1:])
dfTapes.head()

,AMCAR 2017-1,AMCAR 2017-2,AMCAR 2017-3,SDART 2017-1,SDART 2017-2,SDART 2017-3
0,2017\01 January\2017.03.23_AmeriCredit Automob...,2017\03 March\2017.05.02_AmeriCredit Automobil...,2017\06 June\2017.07.01_AmeriCredit Automobile...,2017\01 January\2017.02.15_Santander Drive Aut...,2017\04 April\2017.05.18_Santander Drive Auto ...,2017\08 August\2017.09.07_Santander Drive Auto...
1,2017\02 February\2017.03.23_AmeriCredit Automo...,2017\04 April\2017.06.23_AmeriCredit Automobil...,2017\07 July\2017.08.25_AmeriCredit Automobile...,2017\02 February\2017.03.15_Santander Drive Au...,2017\05 May\2017.06.15_Santander Drive Auto Re...,2017\09 September\2017.10.16_Santander Drive A...
2,2017\03 March\2017.04.24_AmeriCredit Automobil...,2017\05 May\2017.07.24_AmeriCredit Automobile ...,2017\08 August\2017.09.25_AmeriCredit Automobi...,2017\03 March\2017.04.17_Santander Drive Auto ...,2017\06 June\2017.07.17_Santander Drive Auto R...,None
3,2017\04 April\2017.05.23_AmeriCredit Automobil...,2017\06 June\2017.08.23_AmeriCredit Automobile...,None,2017\04 April\2017.05.15_Santander Drive Auto ...,2017\07 July\2017.08.15_Santander Drive Auto R...,None
4,2017\05 May\2017.06.23_AmeriCredit Automobile ...,2017\08 August\2017.09.25_AmeriCredit Automobi...,None,2017\05 May\2017.06.15_Santander Drive Auto Re...,2017\08 August\2017.09.15_Santander Drive Auto...,None


In [22]:
path = directory + '\\' + shelf + '\\' + deal + '\\' + dfTapes[deal][7]
path1 = directory + '\\' + shelf + '\\' + deal + '\\' + dfTapes[deal][6]
path2 = directory + '\\' + shelf + '\\' + deal + '\\' + dfTapes[deal][5]

In [23]:
df = pd.read_csv(path, memory_map=True, low_memory=False)
len(df.index)
df1 = pd.read_csv(path1, memory_map=True, low_memory=False)
df2 = pd.read_csv(path2, memory_map=True, low_memory=False)

In [ ]:
df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio
df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100 #Individual Loan as a Percent of Current month Balance
df['totalCollected'] = df['actualInterestCollectedAmount'] + df['actualPrincipalCollectedAmount'] + df['actualOtherCollectedAmount']
df['principalPaid'] = df['originalLoanAmount'] - df['reportingPeriodActualEndBalanceAmount']

In [24]:
df = df[~df.zeroBalanceCode.isin(['-'])]

In [25]:
cutList = list(df['assetNumber'])
df1 = df1[df1.assetNumber.isin(cutList)]
df2 = df2[df2.assetNumber.isin(cutList)]

In [6]:
path3 = directory + '\\' + shelf + '\\' + deal + '\\' + dfTapes[deal][7]
df3 = pd.read_csv(path3, memory_map=True, low_memory=False)
select = df3.loc[df3['assetNumber'] == '="16490234"']
del df3
select.T

,57763
assetTypeNumber,SDART0201700001
assetNumber,"=""16490234"""
reportingPeriodBeginningDate,08-01-2017
reportingPeriodEndingDate,08-31-2017
originatorName,SC
originationDate,12/2016
originalLoanAmount,13963
originalLoanTerm,72
loanMaturityDate,01/2023
originalInterestRatePercentage,0.1896
